# Etape 3.2 : Analyse des corrélations

In [7]:
import pandas as pd
import numpy as np

df = pd.read_parquet("../output/consommations_enrichies.parquet")
df.shape

(100000, 34)

## Calculer la matrice de corrélation entre :

In [8]:
cols_corr = [
    # Consommation
    "conso_clean",
    "conso_par_m2",
    "conso_par_occupant",

    # Météo
    "temperature_c",
    "humidite_pct",
    "rayonnement_solaire_wm2",
    "vitesse_vent_kmh",

    # Bâtiment
    "surface_m2",
    "nb_occupants_moyen",
    "annee_construction"
]

df_corr = df[cols_corr].dropna()
df_corr.shape


corr_matrix = df_corr.corr(method="pearson")
corr_matrix

corr_matrix.to_csv("../output/matrice_correlation.csv")


## Identifier les corrélations significatives (>0.5 ou <-0.5)

In [9]:
signification = 0.5

corr_significatives = (
    corr_matrix
    .where((corr_matrix >= signification) | (corr_matrix <= -signification))
    .stack()
    .reset_index()
)

corr_significatives.columns = ["variable_1", "variable_2", "correlation"]

# enlever les doublons (A-B / B-A)
corr_significatives = corr_significatives[
    corr_significatives["variable_1"] < corr_significatives["variable_2"]
]

corr_significatives


,variable_1,variable_2,correlation
1,conso_clean,conso_par_m2,0.932531
2,conso_clean,conso_par_occupant,0.920510
3,conso_clean,surface_m2,0.529792
6,conso_par_m2,conso_par_occupant,0.926516


## Analyser l'impact de la température sur la consommation de chauffage

In [10]:
df_chauffage = df[
    df["type_energie"].str.lower().isin(["gaz", "chauffage"])
].dropna(subset=["temperature_c", "conso_clean"])

corr_temp_chauffage = df_chauffage[["temperature_c", "conso_clean"]].corr().iloc[0,1]
corr_temp_chauffage

# Corrélation négative forte → plus il fait froid, plus la consommation augmente

-0.13821304470788143

## Etudier l'effet du rayonnement solaire sur la consommation électrique

In [11]:
df_elec = df[
    df["type_energie"].str.lower().isin(["electricite", "électricité"])
].dropna(subset=["rayonnement_solaire_wm2", "conso_clean"])

corr_solaire_elec = df_elec[["rayonnement_solaire_wm2", "conso_clean"]].corr().iloc[0,1]
corr_solaire_elec

# Plus de rayonnement solaire → légère augmentation de la consommation électrique

0.18221453313487662

# Synthèse des Insights

# Synthèse des insights – Analyse des corrélations

## 1. Corrélations fortes entre indicateurs de consommation

L’analyse met en évidence de fortes corrélations positives entre plusieurs
indicateurs de consommation :

- La consommation brute (`conso_clean`) est fortement corrélée à :
  - la consommation par m² (`conso_par_m2`, corr ≈ 0.93)
  - la consommation par occupant (`conso_par_occupant`, corr ≈ 0.92)
- Une corrélation notable existe également entre la consommation brute et la
  surface du bâtiment (corr ≈ 0.53).

Ces résultats confirment que la taille du bâtiment et l’intensité d’usage
sont des facteurs structurants de la consommation énergétique.

---

## 2. Cohérence des indicateurs normalisés

La forte corrélation entre `conso_par_m2` et `conso_par_occupant`
(corr ≈ 0.93) indique que ces deux indicateurs décrivent un même phénomène
d’intensité énergétique, sous deux angles complémentaires :
- l’occupation humaine
- la surface du bâtiment

Ils constituent donc des métriques pertinentes pour comparer des bâtiments
de tailles et d’usages différents.

---

## 3. Impact de la température sur la consommation de chauffage

La corrélation entre la température extérieure et la consommation énergétique
est faiblement négative (≈ -0.14).

Ce résultat suggère que lorsque la température augmente, la consommation
liée au chauffage tend à diminuer, ce qui est cohérent d’un point de vue
physique. Toutefois, la faible intensité de la corrélation montre que la
température seule n’explique qu’une part limitée de la consommation,
d’autres facteurs jouant un rôle important (isolation, usage, type
d’énergie, comportement des occupants).

---

## 4. Effet du rayonnement solaire sur la consommation électrique

Le rayonnement solaire présente une corrélation faiblement positive avec la
consommation électrique (≈ 0.18).

Ce lien s’explique probablement par une augmentation de l’activité des
bâtiments lors des périodes ensoleillées, notamment via l’utilisation accrue
de la climatisation, de la ventilation et des équipements électriques.
Cependant, l’effet reste modéré, indiquant que le rayonnement solaire n’est
pas un facteur déterminant à lui seul.

---

## 5. Conclusion générale

L’analyse des corrélations met en évidence que :
- Les caractéristiques structurelles des bâtiments (surface, occupation)
  sont les principaux déterminants de la consommation énergétique.
- Les variables météorologiques influencent la consommation, mais de manière
  indirecte et limitée.
- Les indicateurs normalisés (par m², par occupant) sont essentiels pour une
  analyse comparative pertinente entre bâtiments.
